In [1]:
#https://towardsdatascience.com/object-detection-with-10-lines-of-code-d6cb4d86f606
#pip install tensorflow, opencv-python, keras, imageai --upgrade, pip install numpy==1.16
import os
from imageai.Detection import ObjectDetection

import pandas as pd
import geopandas as gpd

from pyproj import Proj
from matplotlib import image as img  
from shapely.geometry import Polygon, Point

execution_path = os.getcwd()
detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath(os.path.join('/Users/Zack/0_thesis_imageai' , "resnet50_coco_best_v2.0.1.h5"))
detector.loadModel()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [2]:
%%time 
#process all images in a folder
input_folder = 'input_folder'
output_folder = 'output_folder'
images = os.listdir(os.path.join(execution_path , input_folder)) #create list of images
#print(len(images), "images in folder")

object_list = pd.DataFrame([])
for i in images:
    detections, extracted_images = detector.detectObjectsFromImage(
        input_image=os.path.join(execution_path , input_folder, i),
        output_image_path=os.path.join(execution_path , output_folder, i), 
        extract_detected_objects=True)
    
    objects = pd.DataFrame(detections) #create df of detections
    objects['image_name'] = i #add image name
    
    #combine dections from all images
    object_list = object_list.append(pd.DataFrame(objects), sort=False) 
object_list

Wall time: 31.3 s


,name,percentage_probability,box_points,image_name
0,person,80.600911,"[342, 346, 410, 438]","image_(19, 37.49039, 22.93714, 37.48995, 22.93..."
1,car,84.235823,"[1121, 452, 1200, 546]","image_(19, 37.49039, 22.93714, 37.48995, 22.93..."
2,car,87.241936,"[1, 423, 100, 616]","image_(19, 37.49039, 22.93714, 37.48995, 22.93..."
3,bus,99.995172,"[80, 192, 1118, 639]","image_(19, 37.49039, 22.93714, 37.48995, 22.93..."
0,bowl,54.498076,"[188, 170, 280, 254]","image_(19, 39.67116, 20.07228, 39.67073, 20.07..."
0,dog,99.986875,"[190, 84, 657, 466]","image_(19, 39.67116, 20.07228, 39.67073, 20.07..."
0,airplane,100.000000,"[64, 44, 643, 412]","image_(19, 40.50401, 22.6655, 40.50356, 22.666..."


In [11]:
#meters per pixel for level 19
res = 0.2986

#meters per pixel for level 18
res = 0.5972

#projection wgs84 to utm
prj = Proj("+proj=utm +zone=34N, +north +ellps=WGS84 +datum=WGS84 +units=m +no_defs")

#create sptatial polygons fro box points
dim_list, poly_list, img_list = [], [], []
top_left_list, btm_left_list, btm_right_list = [], [], []
height_list, width_list = [], []

#convert detections to geopandas polygons
for i in (range(len(object_list))):
   
    #calculate image dimensions
    image = os.path.join(execution_path, output_folder, object_list['image_name'].iloc[i])
    dim = (img.imread(image)).shape
    dim_list.append(dim)
    
    #extract top left lat/lon from image name
    name = object_list['image_name'].iloc[i]
    top_left_lat = float(name.split(", ")[4]) #lat = x
    top_left_lon = float(name.split(", ")[3]) #lon = y
    top_left = (top_left_lat, top_left_lon)
    top_left_list.append(top_left)
    
    #calculate bottom left utm
    dist = res * dim[0] #distance in meters
    top_left_utm = prj(top_left[0], top_left[1]) 
    btm_left_utm = top_left_utm[0], top_left_utm[1] - dist #bottom left utm
    btm_left_list.append(btm_left_utm)

    #convert box points to utm then to lat/lon  
    box = object_list['box_points'].iloc[i]  
    
    #btm_left_utm
    x = btm_left_utm[0]
    y = btm_left_utm[1]
    
    lat_utm = x+box[0]*res, x+box[0]*res, x+box[2]*res, x+box[2]*res 
    lon_utm = y+box[1]*res, y+box[3]*res, y+box[3]*res, y+box[1]*res 
    lat, lon = prj(lat_utm, lon_utm, inverse = True) #convert to lat/lon
    
    poly = Polygon(zip(lat, lon))
    poly_df = gpd.GeoDataFrame(geometry=[poly]) 
    poly_list.append(poly_df)
    
    #calculate box dimensions in meters
    height= (box[3] - box[1]) * res
    width= (box[2] - box[0]) * res
    height_list.append(round(height,2))
    width_list.append(round(width, 2))
    
geom_list = gpd.GeoDataFrame(pd.concat(poly_list, ignore_index=True), crs = {'init': 'epsg:4326'})

In [13]:
#calculate centroid for each detection bounding box
centroid = geom_list['geometry'].centroid
centroid_x, centroid_y = [], []
for i in range(len(centroid)):
    x=centroid[i].xy[0][0]
    y=centroid[i].xy[1][0]
    centroid_x.append(x)
    centroid_y.append(y)

In [15]:
#prepare object list for geometry information
d2 = object_list.reset_index(drop=True) #super important
d2 = d2.drop(columns=['box_points'])
d2['height_m']=height_list
d2['width_m']=width_list
d2['x']=centroid_x
d2['y']=centroid_y

#add polygon infromation to object list
d2_polygons = gpd.GeoDataFrame(d2, geometry = geom_list['geometry'])
d2_polygons
d2_polygons

,name,percentage_probability,image_name,height_m,width_m,x,y,geometry
0,person,80.600911,"image_(19, 37.49039, 22.93714, 37.48995, 22.93...",54.94,40.61,22.940202,37.487713,"POLYGON ((22.93996586886682 37.48746962631547,..."
1,car,84.235823,"image_(19, 37.49039, 22.93714, 37.48995, 22.93...",56.14,47.18,22.945513,37.488202,"POLYGON ((22.94523999381125 37.48795338306093,..."
2,car,87.241936,"image_(19, 37.49039, 22.93714, 37.48995, 22.93...",115.26,59.12,22.938022,37.488435,"POLYGON ((22.93767434173964 37.48792167235698,..."
3,bus,99.995172,"image_(19, 37.49039, 22.93714, 37.48995, 22.93...",266.95,619.89,22.941711,37.487815,"POLYGON ((22.93817558701683 37.48667016669176,..."
4,bowl,54.498076,"image_(19, 39.67116, 20.07228, 39.67073, 20.07...",50.16,54.94,20.074524,39.669640,"POLYGON ((20.0742066017704 39.66941151842646, ..."
5,dog,99.986875,"image_(19, 39.67116, 20.07228, 39.67073, 20.07...",228.13,278.89,20.075847,39.669387,"POLYGON ((20.07423474455175 39.6683463382829, ..."
6,airplane,100.000000,"image_(19, 40.50401, 22.6655, 40.50356, 22.666...",219.77,345.78,22.668561,40.502319,"POLYGON ((22.66649658478572 40.50135931156843,..."


In [6]:
# d2_polygons.to_file('/Users/Zack/Desktop/d2_polygons.geojson', driver='GeoJSON')
# d2_polygons.to_file('/Users/Zack/Desktop/d2_polygons.shp')

In [17]:
#centroid version
centroid_list = gpd.GeoDataFrame(geometry=centroid, crs={'init': 'epsg:4326'})
d2_centroids = gpd.GeoDataFrame(d2, geometry=centroid_list['geometry'])
d2_centroids

,name,percentage_probability,image_name,height_m,width_m,x,y,geometry
0,person,80.600911,"image_(19, 37.49039, 22.93714, 37.48995, 22.93...",54.94,40.61,22.940202,37.487713,POINT (22.94020181936877 37.48771332993602)
1,car,84.235823,"image_(19, 37.49039, 22.93714, 37.48995, 22.93...",56.14,47.18,22.945513,37.488202,POINT (22.94551323511202 37.4882018436909)
2,car,87.241936,"image_(19, 37.49039, 22.93714, 37.48995, 22.93...",115.26,59.12,22.938022,37.488435,POINT (22.93802195587546 37.48843534834985)
3,bus,99.995172,"image_(19, 37.49039, 22.93714, 37.48995, 22.93...",266.95,619.89,22.941711,37.487815,POINT (22.9417106543156 37.48781496770987)
4,bowl,54.498076,"image_(19, 39.67116, 20.07228, 39.67073, 20.07...",50.16,54.94,20.074524,39.669640,POINT (20.07452383412357 39.66964004164571)
5,dog,99.986875,"image_(19, 39.67116, 20.07228, 39.67073, 20.07...",228.13,278.89,20.075847,39.669387,POINT (20.07584663271086 39.66938690898803)
6,airplane,100.000000,"image_(19, 40.50401, 22.6655, 40.50356, 22.666...",219.77,345.78,22.668561,40.502319,POINT (22.66856066485132 40.50231939524626)


In [12]:
#d2_centroids.to_file('/Users/Zack/Desktop/d2_centroids.geojson', driver='GeoJSON')
#d2_centroids.to_file('/Users/Zack/Desktop/d2_centroids.shp')